Imports

In [53]:
import pandas as pd
import yfinance as yf
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch import nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [54]:
ticker_symbol = "AAPL" # "TSLA", "GOOGL", "AAPL"
ticker = yf.Ticker(ticker_symbol)
data = ticker.history(period="2y")
data['Target'] = 0
data.loc[data['Close'].shift(-1) > data['Close'], 'Target'] = 1
data.dropna(inplace=True)
print("length: ", len(data))
data['Daily_Return'] = data['Close'].pct_change()

data['Prev_EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean().shift(1)
data['Prev_RSI'] = (100 - (100 / (1 + (data['Close'].diff().where(data['Close'].diff() > 0, 0).rolling(14).mean() /
                                    (-data['Close'].diff().where(data['Close'].diff() < 0, 0).rolling(14).mean()))))).shift(1)
data['Prev_UpDown'] = (data['Close'].diff() > 0).astype(int)

data = data.dropna()

length:  503


In [55]:
features = data[['Open', 'High', 'Low', 'Close', 'Volume', 'Prev_EMA_10', 'Prev_RSI', 'Prev_UpDown']].values
target = data['Target'].values

# ## Feature Scaling
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(features)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

In [56]:

model_normal = RandomForestClassifier(random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []

for train_index, test_index in kf.split(scaled_features):
    X_train, X_test = scaled_features[train_index], scaled_features[test_index]
    y_train, y_test = target[train_index], target[test_index]
    model_normal.fit(X_train, y_train)
    y_pred = model_normal.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

In [57]:
average_accuracy_full = np.mean(accuracy_scores) * 100
print(f"Classification accuracy score of 10-fold runs (Full Model): {[score * 100 for score in accuracy_scores]}%")
print(f"Average classification accuracy over 10 folds (Full Model): {average_accuracy_full:.2f}%")

Classification accuracy score of 10-fold runs (Full Model): [28.57142857142857, 51.02040816326531, 51.02040816326531, 51.02040816326531, 53.06122448979592, 44.89795918367347, 46.93877551020408, 65.3061224489796, 57.14285714285714, 58.333333333333336]%
Average classification accuracy over 10 folds (Full Model): 50.73%
